# Projet:  
- Réalisation d'une ApplicationDasboard avec Streamlit  
à partir de base de données open source sur les transactions 
de cartes de crédits.
- Lien web vers la source des données:  
[credits_cards_transactions](https://www.kaggle.com/datasets/priyamchoksi/credit-card-transactions-dataset)
- Objectif de notre dasboard:  
Construire un dashboard qui des indicateurs  

## Importation des librairies

- Installation des librairies:  
Exécuter dans la cellule:  %pip install -r requirements.txt  
ou  
Exécuter dans le terminal (prompt ou powershell):  pip install -r requirements.txt

- Importation des librairies

In [27]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import random
import warnings 
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",100)
random.seed(123)

## Chargement des données

In [28]:
# Fonction pour charger les données

def load(file:str, size:int = 10**5, sep:str = ','):
    """--Docstring--
    fonction pour charger les données csv
    tout en optimisant la mémoire du système
    avec les fichiers volumineux. L'option
    size permet de charger les données par
    partition.
    Args:
        file: (string, file.csv or path )
        size: taille de la partition (integer, default = 100000)
              si la mémoire est insuffisante pour importer en un
              bloc.
        sep: séparateur (string, default = <,>)
    """

    try:

        data = pd.read_csv(file, sep=sep, index_col=0)
    
    except MemoryError:

        data = pd.read_csv(file, sep=sep, index_col=0,
                            chunksize = size)
    
    data.index.name = 'index'
    
    return data


In [29]:
# Chargement des données
path = "credit_card_transactions.csv"
financial_data = load(path) 

In [30]:
financial_data.shape

(1296675, 23)

## Exploration des données

### affichage des données

In [31]:
financial_data.head(5)

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,merch_zipcode
index,,,,,,,,,,,,,,,,,,,,,,,
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,NC,28654,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0,28705.0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0,NaN
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,ID,83252,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0,83236.0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,MT,59632,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0,NaN
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,VA,24433,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0,22844.0


### Description sur les types de variables

In [32]:
# Fonction pour décrire le dataframe
    
dictionnaire = {
    'trans_date_trans_time': 'Horodatage format date de la transaction',
    'cc_num': 'Numéro de carte de crédit',
    'merchant': 'Nom du commerce',
    'category' : 'Catégorie de la transaction, exemple: achat épicerie',
    'amt' : 'Montant de la transaction',
    'first': 'Prénom du titulaire de la carte',
    'last': 'Nom de famille du titulaire de la carte',
    'gender' : 'Sexe du titulaire de la carte',
    'street' : 'Rue du titulaire de la carte', 
    'city' :  'Ville du titulaire de la carte', 
    'state' : 'Etat du titulaire de la carte',
    'zip' : 'Code postal du titulaire de la carte', 
    'lat' : "Latitude du lieu d'achat de la transaction",
    'long' : "Longitude du lieu d'achat de la transaction",  
    'city_pop' : 'Population de la ville où la transaction a eu lieu',
    'job' : 'Profession du titulaire de la carte',
    'dob' : 'Date de naissance du titulaire de la carte',
    'trans_num' : 'Numéro de transaction unique',
    'unix_time' : 'Horodatage format Unix de la transaction',
    'merch_lat' : 'Latitude de la position du commerçant',  
    'merch_long': "Longitude de la position du commerçant", 
    'is_fraud' : 'Indicateur indiquant si la transaction est frauduleuse',
    'merch_zipcode' : 'Code postal du commerçant'
    }

def info(data):

    Information = pd.DataFrame({
        'Variables': data.columns,
        'Type': data.dtypes,
        'Unique_values': data.nunique(),
        'NA_counts': data.isna().sum(),
        'NA_percent%':data.isna().mean().round(4)*100,
        }).reset_index(drop=True)

    Information['Description_des_variables'] = Information['Variables'].map(dictionnaire)
    
    return Information

# Application sur financial_data

metadonnees1 = info(financial_data)
metadonnees1

,Variables,Type,Unique_values,NA_counts,NA_percent%,Description_des_variables
0,trans_date_trans_time,object,1274791,0,0.00,Horodatage format date de la transaction
1,cc_num,int64,983,0,0.00,Numéro de carte de crédit
2,merchant,object,693,0,0.00,Nom du commerce
3,category,object,14,0,0.00,"Catégorie de la transaction, exemple: achat ép..."
4,amt,float64,52928,0,0.00,Montant de la transaction
5,first,object,352,0,0.00,Prénom du titulaire de la carte
6,last,object,481,0,0.00,Nom de famille du titulaire de la carte
7,gender,object,2,0,0.00,Sexe du titulaire de la carte
8,street,object,983,0,0.00,Rue du titulaire de la carte
9,city,object,894,0,0.00,Ville du titulaire de la carte


### mise en forme du type des variables

In [33]:
# fonction pour mettre en forme

def conversion(var):

    # conversion des variables de type = <date>
    date_vars = ('trans_date_trans_time', 'dob')

    if var.name in date_vars:
        
        return pd.to_datetime(var, infer_datetime_format = True)


    # conversion des variables type = <objet>
    if (var.dtype=='object') and (var.name not in date_vars):

        # conversion en string
        if var.nunique() > 20:
            return var.astype('string')

        # conversion en catégorie 
        else:
            return var.astype('category')
    
    else:
        return var 


In [34]:
# application de la fonction sur les variables 
financial_data = financial_data.apply(conversion, axis = 0)

### Création de nouvelles variables

- Variables dates

In [35]:
# mettre la variable trans_date_... au format '2020-01-31'
# avec une nouvelle variable

financial_data['date'] = pd.to_datetime(financial_data['trans_date_trans_time']
                                                                  .dt.strftime('%d-%b-%Y'))

# Extraire les périodes sur les transactions : jour, semaine de l'année, mois, heure

financial_data['year'] = (financial_data['trans_date_trans_time']
                                                   .dt.year)

financial_data['day'] = (financial_data['trans_date_trans_time']
                                                   .dt.day_name().astype('category'))

financial_data['month'] = (financial_data['trans_date_trans_time']
                                                   .dt.month_name().astype('category'))

financial_data['hour'] = (financial_data['trans_date_trans_time']
                                                   .dt.strftime('%H.%M')).astype(float)


- Variables Âge, Nom

In [36]:
# Définir la date de référence pour le calcul de l'âge
reference_date = pd.to_datetime('2020-12-31')

# Calcul de l'âge de l'individu à partir de la date de naissance (dob) 
financial_data['age'] = ((reference_date - financial_data['dob']).dt.days/365).astype('int')

# Création de la variable groupe d'âge
financial_data['group_age']=pd.cut(financial_data['age'],
                                   bins=[14,31,46,60,100],
                                   labels=['15-30 ans', '31-45 ans','46-60 ans' ,'+60 ans']
                                  ).astype('category')

# Concaténer first(prénom) et last(nom_de_famille) pour avoir
# nom complet
financial_data['fullname'] = (financial_data['first'] +' '+ financial_data['last']).astype('string')

- Création d'un identifiant unique par individu

In [37]:
financial_data['personid'] = 1 + (financial_data.sort_values(by='fullname')
                                            .groupby(['fullname', 'dob','job']).ngroup())

- Création de la variable Emmeteur de la carte de crédits

In [38]:
# Charger le JSON
with open("iin_ranges.json", "r") as file:
    iin_data = json.load(file)

# Création d'une fonction pour déterminer l'emmeteur des cartes
# en fonction des six premiers chiffres de la carte de crédit

def get_issuer(card_number):
    for network, ranges in iin_data["networks"].items():
        for iin_range in ranges:
            if "-" in iin_range:
                start, end = map(int, iin_range.split("-"))
                if start <= int(card_number[:len(str(start))]) <= end:
                    return network
            else:
                if card_number.startswith(iin_range):
                    return network
    return "Unknown Issuer Network"

# création de la variable issuer identification number = 'iin'

financial_data['iin'] = (financial_data['cc_num'].astype(str)
                                                 .str[:6]
                                                 .apply(get_issuer))

# Regrouper les emmeteurs par grands groupes
def map_iin_group(iin):
    if iin in ['Discover Card', 'Diners Club Intl', 'Diners Club USA & Canada']:
        return 'Discover Financial Services'
    elif iin in ['Mastercard', 'Maestro', 'Maestro UK', 'Switch']:
        return 'Mastercard Group'
    else:
        return iin

# Appliquer la fonction à une nouvelle colonne
financial_data['iin_group'] = financial_data['iin'].apply(map_iin_group)

- Création des variables nombre de transaction, de fraudes,  
montants des transactions, montants des fraudes et taux de fraudes

In [39]:
# liste des variables pour les aggrégations
agglist = ['personid','date']

# Calcul du nombre de transactions journalières
#par personne
financial_data['daily_transactions'] = financial_data.groupby(agglist)['amt'].transform('size')

# Calcul du montant journalier des transactions
# par personne
financial_data['daily_amount$'] = financial_data.groupby(agglist)['amt'].transform('sum')

# Calcul du nombre de fraude journalier sur les transactions 
# par personne
financial_data['daily_number_fraud'] = financial_data.groupby(agglist)['is_fraud'].transform('sum')

# Calcul du cout = montant journalier des fraudes sur transactions 
# par personne
financial_data['daily_fraud_amount$'] = (financial_data[financial_data['is_fraud']==1]
                                                            .groupby(agglist)['amt']
                                                            .transform('sum')
                                      )
financial_data['daily_fraud_amount$'].fillna(0, inplace = True)

# calcul du taux de fraude journalier par personne

financial_data['daily_fraud_rate'] = ((financial_data['daily_number_fraud']/
                                       financial_data['daily_transactions'])
                                       .round(4)*100
                                      )


liste = ['personid','date', 'year', 'month', 'day', 'fullname','dob','age','group_age','gender','job','cc_num',
         'daily_transactions','amt','daily_amount$','daily_number_fraud','daily_fraud_amount$',
         'daily_fraud_rate','category','iin', 'iin_group','city','street','city_pop','long','lat','state']
# Affichage des données
financial_data[liste].sort_values(by = agglist, ignore_index=True).head(5)

,personid,date,year,month,day,fullname,dob,age,group_age,gender,job,cc_num,daily_transactions,amt,daily_amount$,daily_number_fraud,daily_fraud_amount$,daily_fraud_rate,category,iin,iin_group,city,street,city_pop,long,lat,state
0,1,2019-01-01,2019,January,Tuesday,Aaron Murray,1974-12-23,46,31-45 ans,M,Tourist information centre manager,376028110684021,7,89.11,889.67,0,0.0,0.0,grocery_pos,American Express,American Express,Meadville,624 Hale Springs Apt. 572,964,-93.3014,39.7795,MO
1,1,2019-01-01,2019,January,Tuesday,Aaron Murray,1974-12-23,46,31-45 ans,M,Tourist information centre manager,376028110684021,7,68.85,889.67,0,0.0,0.0,gas_transport,American Express,American Express,Meadville,624 Hale Springs Apt. 572,964,-93.3014,39.7795,MO
2,1,2019-01-01,2019,January,Tuesday,Aaron Murray,1974-12-23,46,31-45 ans,M,Tourist information centre manager,376028110684021,7,47.96,889.67,0,0.0,0.0,kids_pets,American Express,American Express,Meadville,624 Hale Springs Apt. 572,964,-93.3014,39.7795,MO
3,1,2019-01-01,2019,January,Tuesday,Aaron Murray,1974-12-23,46,31-45 ans,M,Tourist information centre manager,376028110684021,7,77.51,889.67,0,0.0,0.0,health_fitness,American Express,American Express,Meadville,624 Hale Springs Apt. 572,964,-93.3014,39.7795,MO
4,1,2019-01-01,2019,January,Tuesday,Aaron Murray,1974-12-23,46,31-45 ans,M,Tourist information centre manager,376028110684021,7,461.28,889.67,0,0.0,0.0,travel,American Express,American Express,Meadville,624 Hale Springs Apt. 572,964,-93.3014,39.7795,MO


### Extraction des données

In [40]:
# extraction des données néttoyées
financial_data[liste].to_csv('financial_clean.csv', chunksize = 10**5, index=False)

# Sous base de données individus
# l'unité d'observation est la transaction journalière d'une personne
individual_data = (financial_data[liste].drop(['group_age','city_pop','amt'], axis = 1)
                                        .drop_duplicates(subset = agglist)
                                        .sort_values(by = agglist,ignore_index = True)    
                   )
individual_data.to_csv('individual_data.csv', index=False)

# extraction des métadonnées sur les variables d'origine
metadonnees1.to_csv('metadonnees1.csv', index=False)

In [41]:
# extraction des métadonnées sur les nouvelles variables calculées
dico = {'personid':'Identifiant unique du détenteur de la carte',
        'date':'la nouvelle date au format yyyy-mm-dd',
        'year':'Année',
        'day':'les 7 jours de la semaine, ex: Lundi',
        'month':'le mois, ex: Janvier',
        'hour':'heure au format HH.Minute',
        'age':'âge en années exactes du détenteur de la carte',
        'group_age':"Groupe d'âges",
        'fullname':'nom complet du détenteur de la carte',
        'iin':'les emmeteurs individuels de cartes de crédits',
        'iin_group':'les emmeteurs par grands groupes, ex: Master, VISA, etc',
        'daily_transactions':'total journalier du nombre de transactions sur la carte du détenteur',
        'daily_amount$':'total journalier des montants de transactions sur la carte du détenteur',
        'daily_number_fraud':'total journalier du nombre de fraudes sur la carte du détenteur',
        'daily_fraud_amount$':"""total journalier des montants de 
                           transactions frauduleuses  sur la carte du détenteur""",
        'daily_fraud_rate':'taux journalier de fraudes sur la carte du détenteur'}

In [42]:
originvar = metadonnees1['Variables'].unique().tolist()
newvar = financial_data.drop(columns=originvar).columns.tolist()
metadonnees2 = info(financial_data[newvar])
metadonnees2['Description_des_variables']=metadonnees2['Variables'].map(dico)
metadonnees2.to_csv('metadonnees2.csv', index=False)

- fonction pour tableau Croisé de Statistiques descriptives

In [43]:
def cross_stat(data:pd.DataFrame, catlist:list, statvars:list):

    """--Docstring--
    fonction pour réaliser des statistiques
    sur des variables croisées.
    
    Args:
         data: le dataframe
         catlist: (list) variables catégorielles
         statvars: (list) variables continues
    
    return a dataframe
    """
    # initialisation d'un tableau vide
    table = pd.DataFrame()
    
    # Création de dictionnaire pour les indexes
    index_mapping = {value: cat for cat in catlist for value in data[cat].unique()}

    
    for var in statvars:

        for cat in catlist:
            
            X = data.groupby(by=cat)[var].agg(min = 'min',
                                              max = 'max',
                                              mean = 'mean',
                                              st_deviation = 'std',
                                              quartile1 = lambda x: x.quantile(0.25),
                                              median = 'median',
                                              quartile3 = lambda x: x.quantile(0.75),
                                              category_size = 'count')

            table = pd.concat([table,X], axis = 0, ignore_index = False)
    
        table.reset_index(names = 'valeurs', inplace = True)
        table['Categories'] = table['valeurs'].map(index_mapping)
        table.set_index(['Categories','valeurs'], inplace = True)
    
        print(f'Tableau Statistiques croisées sur: {var}')
        display(table.round(2))
        print('\n\n')
